In [2]:
#imports
import pandas as pd
import numpy as np
import nltk
import xgboost
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from collections import Counter

from sklearn import model_selection
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras import regularizers , optimizers
from sklearn import preprocessing,metrics,naive_bayes,svm,linear_model,ensemble
from sklearn.feature_extraction.text import TfidfVectorizer

from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing import text, sequence
import matplotlib.pyplot as plt
import numpy as np

Using TensorFlow backend.


In [5]:
#reading critical dataset 
critical = pd.read_csv("sample_data/critical-findings-sample-data.csv",encoding = 'utf-8')

#reading non-critical dataset
non_critical = pd.read_csv("sample_data/non-critical-findings-sample-data.csv",encoding = 'utf-8')

In [6]:
#replacing None with No critical Finding
non_critical['Critical_Finding'] = non_critical['Critical_Finding'].replace(regex='None', value='No Critical Finding')

#replacing NaN with No Category
non_critical['Category'] = 'No Category'


In [7]:
#merging critical and non-critical datasets
findings = pd.concat([critical,non_critical])

#restting index after merge
findings=findings.reset_index(drop=True) 

# Preprocessing

In [8]:
#converts the data column to lower case
findings['Data'] = findings['Data'].astype(str).str.lower()

#since xray is found in glove replacing x-ray with xray
findings['Data'] = findings['Data'].str.replace('x-ray','xray') 

#html tags replace
tag_replace = '<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6})' 
findings['Data'] = findings['Data'].str.replace(tag_replace, ' ') 

#replacing punctuations with whitespace 
pattern = '[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
findings['Data'] = findings['Data'].str.replace(pattern, ' ')           

#Numbers removal
findings['Data'] = findings['Data'].replace('\d+', '', regex=True) 

#replaces many spaces with a single space
findings['Data'] = findings['Data'].replace('\s+', ' ', regex=True)


In [9]:
# split the dataset into training and validation datasets 
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(findings['Data'], findings['Category'])

# label encode the target variable 
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

# TF-IDF

In [10]:

# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
tfidf_vect.fit(findings['Data'])
xtrain_tfidf =  tfidf_vect.transform(train_x)
xvalid_tfidf =  tfidf_vect.transform(valid_x)

# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram.fit(findings['Data'])
xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
tfidf_vect_ngram_chars.fit(findings['Data'])
xtrain_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(train_x) 
xvalid_tfidf_ngram_chars =  tfidf_vect_ngram_chars.transform(valid_x) 

# GloVe

In [11]:
# load the pre-trained word-embedding vectors 
embeddings_index = {}
for i, line in enumerate(open('glove.txt','r', encoding = 'utf-8')):
    values = line.split()
    embeddings_index[values[0]] = np.asarray(values[1:], dtype='float32')

# create a tokenizer 
token = text.Tokenizer()
token.fit_on_texts(findings['Data'])
word_index = token.word_index

# convert text to sequence of tokens and pad them to ensure equal length vectors 
train_seq_x = sequence.pad_sequences(token.texts_to_sequences(train_x), maxlen=737)
valid_seq_x = sequence.pad_sequences(token.texts_to_sequences(valid_x), maxlen=737)

# create token-embedding mapping
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Models

In [38]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label , epochs = 10)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, valid_y)

## Naive Bayes 

In [13]:
# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("NB, WordLevel TF-IDF: ", accuracy)

# Naive Bayes on Ngram Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("NB, N-Gram Vectors: ", accuracy)

# Naive Bayes on Character Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("NB, CharLevel Vectors: ", accuracy)

NB, WordLevel TF-IDF:  0.564
NB, N-Gram Vectors:  0.692
NB, CharLevel Vectors:  0.516


## Logistic Regression

In [14]:
# Linear Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("LR, WordLevel TF-IDF: ", accuracy)

# Linear Classifier on Ngram Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("LR, N-Gram Vectors: ", accuracy)

# Linear Classifier on Character Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("LR, CharLevel Vectors: ", accuracy)

C:\Users\divya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\divya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LR, WordLevel TF-IDF:  0.732
LR, N-Gram Vectors:  0.708
LR, CharLevel Vectors:  0.62


## SVM

In [15]:
# SVM on Word Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("SVM, WordLevel TF-IDF: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("SVM, N-Gram Vectors: ", accuracy)

# SVM on Ngram Level TF IDF Vectors
accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("SVM, CharLevel Vectors: ", accuracy)

C:\Users\divya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM, WordLevel TF-IDF:  0.512
SVM, N-Gram Vectors:  0.512
SVM, CharLevel Vectors:  0.512


## Random Forests 

In [16]:

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf, train_y, xvalid_tfidf)
print ("RF, WordLevel TF-IDF: ", accuracy)

# RF on Ngram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
print ("RF, N-Gram Vectors: ", accuracy)

# RF on Ngram Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram_chars, train_y, xvalid_tfidf_ngram_chars)
print ("RF, CharLevel Vectors: ", accuracy)

C:\Users\divya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\divya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RF, WordLevel TF-IDF:  0.928
RF, N-Gram Vectors:  0.92
RF, CharLevel Vectors:  0.928


C:\Users\divya\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


## Extreme Gradient Boosting

In [17]:
# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf.tocsc(), train_y, xvalid_tfidf.tocsc())
print ("Xgb, WordLevel TF-IDF: ", accuracy)

# Extereme Gradient Boosting on Ngram Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram.tocsc(), train_y, xvalid_tfidf_ngram.tocsc())
print ("Xgb, N-Gram Vectors: ", accuracy)

# Extereme Gradient Boosting on Character Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), xtrain_tfidf_ngram_chars.tocsc(), train_y, xvalid_tfidf_ngram_chars.tocsc())
print ("Xgb, CharLevel Vectors: ", accuracy)

Xgb, WordLevel TF-IDF:  0.968
Xgb, N-Gram Vectors:  0.952
Xgb, CharLevel Vectors:  0.972


## Shallow Neural Network

In [37]:
def create_model_architecture(input_size):
    # create input layer 
    input_layer = layers.Input((input_size, ), sparse=True)
    
    # create hidden layer
    hidden_layer = layers.Dense(100, activation="relu")(input_layer)
    
    # create output layer
    output_layer = layers.Dense(1, activation="sigmoid")(hidden_layer)

    classifier = models.Model(inputs = input_layer, outputs = output_layer)
    classifier.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    return classifier 

classifier = create_model_architecture(xtrain_tfidf_ngram.shape[1])

accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
print ("NN, Ngram Level TF IDF Vectors",  accuracy)

# classifier = create_model_architecture(train_seq_x.shape[1])
# accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
# print ("NN, Word Embeddings",  accuracy)

Epoch 1/1
750/750 [==============================] - 2s 2ms/step - loss: -1.2482
NN, Ngram Level TF IDF Vectors 0.1


## CNN with GloVe

In [39]:
def create_cnn():
    # Add an Input Layer
    input_layer = layers.Input((737, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_cnn()
accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)




# accuracy = train_model(classifier, xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram, is_neural_net=True)
# print ("CNN, Ngram Level TF IDF Vectors",  accuracy)


Epoch 1/10
750/750 [==============================] - 9s 12ms/step - loss: -26.1859
Epoch 2/10
750/750 [==============================] - 7s 9ms/step - loss: -38.6039
Epoch 3/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6347
Epoch 4/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6321
Epoch 5/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6308
Epoch 6/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6342
Epoch 7/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6181
Epoch 8/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6366
Epoch 9/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6371
Epoch 10/10
750/750 [==============================] - 7s 10ms/step - loss: -38.6410
CNN, Word Embeddings 0.1


## rnn & lstm

In [40]:
def create_rnn_lstm():
    # Add an Input Layer
    input_layer = layers.Input((737, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.LSTM(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rnn_lstm()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-LSTM, Word Embeddings",  accuracy)

Epoch 1/10
750/750 [==============================] - 17s 22ms/step - loss: -18.1241
Epoch 2/10
750/750 [==============================] - 14s 19ms/step - loss: -38.5753
Epoch 3/10
750/750 [==============================] - 15s 20ms/step - loss: -38.6018
Epoch 4/10
750/750 [==============================] - 15s 20ms/step - loss: -38.6018
Epoch 5/10
750/750 [==============================] - 15s 20ms/step - loss: -38.6018
Epoch 6/10
750/750 [==============================] - 15s 21ms/step - loss: -38.6018
Epoch 7/10
750/750 [==============================] - 18s 23ms/step - loss: -38.6018
Epoch 8/10
750/750 [==============================] - 16s 21ms/step - loss: -38.6018
Epoch 9/10
750/750 [==============================] - 15s 20ms/step - loss: -38.6018
Epoch 10/10
750/750 [==============================] - 15s 20ms/step - loss: -38.6018
RNN-LSTM, Word Embeddings 0.1


## RNN & GRU 

In [41]:
def create_rnn_gru():
    # Add an Input Layer
    input_layer = layers.Input((737, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the GRU Layer
    lstm_layer = layers.GRU(100)(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy', )
    
    return model

classifier = create_rnn_gru()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-GRU, Word Embeddings",  accuracy)

Epoch 1/10
750/750 [==============================] - 17s 22ms/step - loss: -13.8944
Epoch 2/10
750/750 [==============================] - 16s 21ms/step - loss: -38.4380
Epoch 3/10
750/750 [==============================] - 15s 20ms/step - loss: -38.5815
Epoch 4/10
750/750 [==============================] - 15s 19ms/step - loss: -38.5963
Epoch 5/10
750/750 [==============================] - 15s 21ms/step - loss: -38.5926
Epoch 6/10
750/750 [==============================] - 15s 20ms/step - loss: -38.5963
Epoch 7/10
750/750 [==============================] - 20s 27ms/step - loss: -38.6018
Epoch 8/10
750/750 [==============================] - 23s 31ms/step - loss: -38.6018
Epoch 9/10
750/750 [==============================] - 22s 30ms/step - loss: -38.6018
Epoch 10/10
750/750 [==============================] - 22s 30ms/step - loss: -38.5948
RNN-GRU, Word Embeddings 0.1


## Bi-directional  RNN

In [42]:
def create_bidirectional_rnn():
    # Add an Input Layer
    input_layer = layers.Input((737, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)

    # Add the LSTM Layer
    lstm_layer = layers.Bidirectional(layers.GRU(100))(embedding_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(lstm_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_bidirectional_rnn()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("RNN-Bidirectional, Word Embeddings",  accuracy)

Epoch 1/10
750/750 [==============================] - 38s 50ms/step - loss: -12.8281
Epoch 2/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6013
Epoch 3/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6018
Epoch 4/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6018
Epoch 5/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6018
Epoch 6/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6018
Epoch 7/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6018
Epoch 8/10
750/750 [==============================] - 30s 40ms/step - loss: -38.6018
Epoch 9/10
750/750 [==============================] - 31s 41ms/step - loss: -38.6018
Epoch 10/10
750/750 [==============================] - 31s 42ms/step - loss: -38.6018
RNN-Bidirectional, Word Embeddings 0.1


## RCNN 

In [43]:
def create_rcnn():
    # Add an Input Layer
    input_layer = layers.Input((737, ))

    # Add the word embedding Layer
    embedding_layer = layers.Embedding(len(word_index) + 1, 300, weights=[embedding_matrix], trainable=False)(input_layer)
    embedding_layer = layers.SpatialDropout1D(0.3)(embedding_layer)
    
    # Add the recurrent layer
    rnn_layer = layers.Bidirectional(layers.GRU(50, return_sequences=True))(embedding_layer)
    
    # Add the convolutional Layer
    conv_layer = layers.Convolution1D(100, 3, activation="relu")(embedding_layer)

    # Add the pooling Layer
    pooling_layer = layers.GlobalMaxPool1D()(conv_layer)

    # Add the output Layers
    output_layer1 = layers.Dense(50, activation="relu")(pooling_layer)
    output_layer1 = layers.Dropout(0.25)(output_layer1)
    output_layer2 = layers.Dense(1, activation="sigmoid")(output_layer1)

    # Compile the model
    model = models.Model(inputs=input_layer, outputs=output_layer2)
    model.compile(optimizer=optimizers.Adam(), loss='binary_crossentropy')
    
    return model

classifier = create_rcnn()

accuracy = train_model(classifier, train_seq_x, train_y, valid_seq_x, is_neural_net=True)
print ("CNN, Word Embeddings",  accuracy)

Epoch 1/10
750/750 [==============================] - 18s 23ms/step - loss: -27.4945
Epoch 2/10
750/750 [==============================] - 13s 17ms/step - loss: -38.5494
Epoch 3/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6238
Epoch 4/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6012
Epoch 5/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6289
Epoch 6/10
750/750 [==============================] - 13s 17ms/step - loss: -38.5971
Epoch 7/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6235
Epoch 8/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6319
Epoch 9/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6247
Epoch 10/10
750/750 [==============================] - 13s 17ms/step - loss: -38.6426
CNN, Word Embeddings 0.1
